In [2]:
#--------------
#INITIALIZATION
#--------------

%matplotlib inline

import sys, os
import numpy as np
import math
from scipy.integrate import dblquad
import matplotlib.pyplot as plt
import matplotlib.ticker
from matplotlib import rc, rcParams
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Make use of TeX\ufeff
rc('text',usetex=True)
# Change all fonts to 'Computer Modern'
rc('font',**{'size':14, 'family':'serif','serif':['Times New Roman']})
rc('xtick.major', size=5, pad=7)
rc('xtick', labelsize=15)
rc('ytick.major', size=5, pad=7)
rc('ytick', labelsize=15)

# this needs to point to the folder where darkelf.py is stored
work_dir = os.getcwd()
sys.path.append(work_dir+"/..")
plotdir=work_dir+"/plots/"

from darkelf import darkelf, targets

In [3]:
#----
#DATA
#----

Si = darkelf(target='Si',filename='Si_mermin.dat',phonon_filename="Si_epsphonon_data6K.dat")

Ge = darkelf(target='Ge',filename='Ge_mermin.dat',phonon_filename="Ge_epsphonon_data2K.dat")

Al = darkelf(target='Al',filename='Al_mermin.dat')

SiO2 = darkelf(target='SiO2',filename='SiO2_mermin.dat')

 .... Loading files for Si
Loaded Si_mermin.dat for epsilon in electron regime
Loaded Si_epsphonon_data6K.dat for epsilon in phonon regime
 .... Loading files for Ge
Loaded Ge_mermin.dat for epsilon in electron regime
Loaded Ge_epsphonon_data2K.dat for epsilon in phonon regime
 .... Loading files for Al
Loaded Al_mermin.dat for epsilon in electron regime
Warning! eps for phonon frequencies not loaded. Need to set phonon_filename 
Warning! Atomic Migdal calculation not present


In [4]:
#--------------------
#SINGLE MATERIAL PLOT
#--------------------

#Velocities [km/s]
v_0   = 230
v_e   = 240
v_esc = 600
c     = 299792.458
v_max = (v_esc + v_e)/c

#DM mass [eV]
m_X = 1e7
Si.update_params(mX = m_X,mediator = 'massless')
Ge.update_params(mX = m_X,mediator = 'massless')
Al.update_params(mX = m_X,mediator = 'massless')

#Truncated Maxwell-Boltzmann Distribution (Appendix B heterostructures)
N_0 = np.pi**(3/2)*v_0**2*(v_0*math.erf(v_esc/v_0) - \
      2*v_esc/np.sqrt(np.pi)*np.exp(-v_esc**2/v_0**2))
a = (np.pi**(3/2)*v_0**3)/(4*v_e*N_0)

def p1(v_z):
        return (a*(math.erf((v_e-v_z)/v_0) + math.erf((v_e+v_z)/v_0) - \
                (np.pi*v_0**2/N_0)*np.exp(-v_esc**2/v_0**2)))*(v_z < v_esc - v_e) + \
                (a*(math.erf(v_esc/v_0) + math.erf((v_e-v_z)/v_0)) - \
                (np.pi*v_0**2)/(2*N_0)*(v_e + v_esc - v_z)/v_e*np.exp(-v_esc**2/v_0**2)) * \
                (v_esc - v_e < v_z < v_esc + v_e)

#I(w,k) for massless mediator
def I(w,k):
      return (w/k*p1(w/k + k/(2*m_X)))/w

#w and k limits
w_min = 0
w_max = (v_max**2 * m_X / 2)
k_min = lambda w: (v_max*m_X - np.sqrt(v_max**2 * m_X**2 - 2 * m_X * w))
k_max = lambda w: (v_max*m_X + np.sqrt(v_max**2 * m_X**2 - 2 * m_X * w))

eVtoInvS = 1.5192674e15 # eV to 1/s
yeartosec = 86400 * 365
eVtoInvYr = eVtoInvS * yeartosec

#Solid angle
solid = 1/(2*np.pi)**3#1/(2*np.pi**2)

#Integral of I(w,k)ELF(w,k)
def integrand():
      f = lambda k, w: I(w,k)*Si.elf(w,k,method='grid') / eVtoInvYr
      return dblquad(f,w_min,w_max,k_min,k_max)[0] * eVtoInvYr

#Gamma-opt
def gamma_opt(m_X):
      return 0.68/(4*math.pi)*m_X*v_0/c

#Gamma-bar 
def gamma_bar():
      return 2*integrand()*solid*c

#Gamma-ratio plot
mX = np.logspace(5,9,20) #0.01 - 1000 MeV
gamma_ratio = []
step = 0
for x in mX:
      step  = step + 1
      print('Step:',step)
      m_X   = x
      w_min = 1.12
      w_max = (v_max**2 * m_X / 2)
      k_min = lambda w: (v_max*m_X - np.sqrt(v_max**2 * m_X**2 - 2 * m_X * w))
      k_max = lambda w: (v_max*m_X + np.sqrt(v_max**2 * m_X**2 - 2 * m_X * w))
      Si.update_params(mX = m_X,mediator = 'massless')
      gamma_ratio.append(gamma_opt(x)/gamma_bar())

mX = mX/1e6

plt.loglog(mX,gamma_ratio,color = 'red',label = 'Si')
plt.xlabel(r'$m_\chi$')
plt.ylabel(r'$\Gamma_{opt}/\overline{\Gamma}$')
plt.legend()
plt.show

#----------------
#VARIABLE TESTING
#----------------
#print('For mX       =',m_X,'eV')
#print('---------------------------')
#print('Gamma-bar    =',gamma_bar())
#print('Gamma_opt    =',gamma_opt(m_X))
#print('integrand()  =',integrand())
#print('w_min        =',w_min)
#print('w_max        =',w_max)
#print('k_min(w_min) =',k_min(w_min))
#print('k_min(w_max) =',k_min(w_max))
#print('k_max(w_min) =',k_max(w_min))
#print('k_max(w_max) =',k_max(w_max))
#print('v_max*m_X    =',v_max*m_X)
#print('gamma_opt/gamma_bar = ', gamma_opt(m_X)/gamma_bar())


NameError: name 'm_X' is not defined

In [ ]:
#ELF CONTOUR PLOT
wmin,wmax = 0,30
kmin,kmax = 500,4000

omega_vec = np.linspace(wmin,wmax,500)
k_vec     = np.linspace(kmin,kmax,500)

np.seterr(divide='ignore', invalid='ignore')
[K,W] = np.meshgrid(k_vec,omega_vec)
X = np.zeros((len(K), len(K[0])))
Y = np.zeros((len(K), len(K[0])))
contour_func = np.zeros((len(K), len(K[0])))

for i in range(len(K)):
        for j in range(len(K[0])):
                contour_func[i][j] = Si.elf(W[i][j],K[i][j],method='Lindhard')
                X[i][j] = K[i][j]
                Y[i][j] = W[i][j] 

plt.contourf(X,Y,contour_func,9)
plt.colorbar()
plt.show()

In [ ]:
#----------------------
#MULTIPLE MATERIAL PLOT
#----------------------
from IPython.display import clear_output
#Integral of I(w,k)ELF(w,k)
def Si_integrand():
      f = lambda k, w: I(w,k)*Si.elf(w,k,method='grid') / eVtoInvYr
      return dblquad(f,w_min_si,w_max,k_min,k_max)[0] * eVtoInvYr

def Ge_integrand():
      f = lambda k, w: I(w,k)*Ge.elf(w,k,method='grid') / eVtoInvYr
      return dblquad(f,w_min,w_max,k_min,k_max)[0] * eVtoInvYr

def Al_integrand():
      f = lambda k, w: I(w,k)*Al.elf(w,k,method='grid') / eVtoInvYr
      return dblquad(f,w_min_al,w_max,k_min,k_max)[0] * eVtoInvYr

def SiO2_integrand():
      f = lambda k, w: I(w,k)*SiO2.elf(w,k,method='grid') / eVtoInvYr
      return dblquad(f,w_min_al,w_max,k_min,k_max)[0] * eVtoInvYr

#Gamma-bar 
def Si_gamma_bar():
      return 2*Si_integrand()*solid*c

def Ge_gamma_bar():
      return 2*Ge_integrand()*solid*c

def Al_gamma_bar():
      return 2*Al_integrand()*solid*c

def SiO2_gamma_bar():
      return 2*SiO2_integrand()*solid*c


#Produces a vector gamma_opt/gamma_bar of all different mX
mX = np.logspace(5,9,100) #0.01 - 1000 MeV
Si_gamma_ratio   = []
Al_gamma_ratio   = []
SiO2_gamma_ratio = []

step = 0
for x in mX:
      step = step + 1
      clear_output(wait=True)
      print('Working... (',step,'/',len(mX),')',sep='')
      
      m_chi = x
      w_min_al = .01
      w_min_si = 1.12
      w_max = (v_max**2 * m_chi / 2)
      k_min = lambda w: (v_max*m_chi - np.sqrt(v_max**2 * m_chi**2 - 2 * m_chi * w))
      k_max = lambda w: (v_max*m_chi + np.sqrt(v_max**2 * m_chi**2 - 2 * m_chi * w))

      Si.update_params(mX = m_chi,mediator = 'massless')
      Al.update_params(mX = m_chi,mediator = 'massless')
      SiO2.update_params(mX=m_chi,mediator = 'massless')

      Si_gamma_ratio.append(gamma_opt(m_chi)/Si_gamma_bar())
      Al_gamma_ratio.append(gamma_opt(m_chi)/Al_gamma_bar())
      SiO2_gamma_ratio.append(gamma_opt(m_chi)/SiO2_gamma_bar())

mX = mX/1e6

plt.loglog(mX,Si_gamma_ratio,color = 'red',label = 'Si')
plt.loglog(mX,Al_gamma_ratio,color = 'green',label = 'Al')
plt.loglog(mX,SiO2_gamma_ratio,color = 'yellow',label = 'SiO2')
plt.xlabel(r'$m_\chi \textrm{/MeV}$')
plt.ylabel(r'$\Gamma_{opt}/\overline{\Gamma}$')
plt.legend()
plt.xlim([mX[0],mX[len(mX)-1]])
plt.ylim([1,1e5])
plt.show
#plt.savefig("NAME HERE.pdf",bbox_inches='tight')